In [ ]:
import cv2
from crop_white_space.image_processing import otsus_thresholding, canny_edge_detection, find_contour_bounds, morphological_operations
from crop_white_space.utils import show_or_save_image
from crop_white_space.config import read_config

In [12]:
def main():
    config = read_config()

    # Extract parameters from config
    otsu_params = config.get('otsu', {})
    canny_params = config.get('canny', {})
    morph_params = config.get('morphological_operations', {})

    path_raw = r"../data/raw/img.png"
    path_internim = r"../data/interim/img.png"
    path_preproceesed = r"../data/processed/img.png"

    # Read the image
    image = cv2.imread(path_raw)
    if image is None:
        print(f"Error: Unable to read image {path_raw}")
        return

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's thresholding
    otsu_thresh = otsus_thresholding(gray, **otsu_params)
    show_or_save_image(otsu_thresh, "Otsu Threshold",
                       path=path_internim,
                       filename="otsu_thresh.png")

    # Optionally apply morphological operations
    morphed = morphological_operations(otsu_thresh, **morph_params)
    show_or_save_image(morphed, "Morphological Operations",
                       path=path_internim,
                       filename="morphed.png")

    # Canny edge detection
    edges = canny_edge_detection(morphed, **canny_params)
    show_or_save_image(edges, "Canny Edges",
                       path=path_internim,
                       filename="canny_edges.png")

    # Find contour bounds
    contour_bounds = find_contour_bounds(edges)
    if contour_bounds:
        x1, y1, x2, y2 = contour_bounds
        cropped_image = image[y1:y2, x1:x2]
        show_or_save_image(cropped_image, "Cropped Image",
                       path=path_internim,
                       filename="cropped_image.png")

        # Save the final cropped image
        cv2.imwrite(path_preproceesed, cropped_image)
    else:
        print("No significant contours found.")

In [13]:
if __name__ == "__main__":
    main()